In [2]:
#identifying artists with different streams distribution

In [17]:
import pandas as pd

In [18]:
df = pd.read_csv("/content/drive/MyDrive/DV_A3/Final Datasets/merged.csv")
df['Total Spotify Streams'] = df.groupby('Artist')['Spotify Streams'].transform('sum')
df['Total YouTube Views'] = df.groupby('Artist')['YouTube Views'].transform('sum')
df['Total TikTok Views'] = df.groupby('Artist')['TikTok Views'].transform('sum')
print(df)

                   track_id               Artist  \
0    0IGUXY4JbK18bu9oD4mPIm             (G)I-DLE   
1    6Pgmqg15yVexuOgtzuxwoX             347aidan   
2    27XMmSWSOXJ7eVb40tH93T             347aidan   
3    2iUXsYOEPhVqEBwsqP70rE  5 Seconds of Summer   
4    26wLOs3ZuHJa2Ihhx6QIE6  5 Seconds of Summer   
..                      ...                  ...   
626  6qUabsNxudatEFedWGHoDh         Travis Scott   
627  7t2bFihaDvhIrd2gn2CWJO        Ty Dolla $ign   
628  2LskIZrCeLxRvCiGP8gxlh         XXXTENTACION   
629  39QJ7KkN3L8LLqQqsO85W1    Yo Yo Honey Singh   
630  1IIKrJVP1C9N7iPtG6eOsK           Young Thug   

                                            album_name  \
0                                          I NEVER DIE   
1                                   Dancing in My Room   
2                                   TROUBLED MEMORIES!   
3                                  Youngblood (Deluxe)   
4                            13 Reasons Why (Season 3)   
..                         

In [19]:
df['Performs well on Spotify'] = 0
df['Performs well on TikTok'] = 0
df['Performs well on YouTube'] = 0


# Iterate through each row to apply the conditions
for idx, row in df.iterrows():
    total_views = row['Spotify Streams'] + row['TikTok Views'] + row['YouTube Views']

    # Calculate percentages
    spotify_percentage = (row['Spotify Streams'] / total_views) * 100 if total_views > 0 else 0
    tiktok_percentage = (row['TikTok Views'] / total_views) * 100 if total_views > 0 else 0
    youtube_percentage = (row['YouTube Views'] / total_views) * 100 if total_views > 0 else 0

    # Apply the conditions
    if spotify_percentage >= 55:
        df.at[idx, 'Performs well on Spotify'] = 1
    elif tiktok_percentage >= 55:
        df.at[idx, 'Performs well on TikTok'] = 1
    elif youtube_percentage >= 55:
        df.at[idx, 'Performs well on YouTube'] = 1
    elif (spotify_percentage + tiktok_percentage >= 75):
        df.at[idx, 'Performs well on Spotify'] = 1
        df.at[idx, 'Performs well on TikTok'] = 1
    elif (spotify_percentage + youtube_percentage >= 75):
        df.at[idx, 'Performs well on Spotify'] = 1
        df.at[idx, 'Performs well on YouTube'] = 1
    elif (tiktok_percentage + youtube_percentage >= 75):
        df.at[idx, 'Performs well on TikTok'] = 1
        df.at[idx, 'Performs well on YouTube'] = 1
    elif spotify_percentage >= 22 and tiktok_percentage >= 22 and youtube_percentage >= 22:
        df.at[idx, 'Performs well on Spotify'] = 1
        df.at[idx, 'Performs well on TikTok'] = 1
        df.at[idx, 'Performs well on YouTube'] = 1


In [6]:
print(df)

                   track_id               Artist  \
0    0IGUXY4JbK18bu9oD4mPIm             (G)I-DLE   
1    6Pgmqg15yVexuOgtzuxwoX             347aidan   
2    27XMmSWSOXJ7eVb40tH93T             347aidan   
3    2iUXsYOEPhVqEBwsqP70rE  5 Seconds of Summer   
4    26wLOs3ZuHJa2Ihhx6QIE6  5 Seconds of Summer   
..                      ...                  ...   
626  6qUabsNxudatEFedWGHoDh         Travis Scott   
627  7t2bFihaDvhIrd2gn2CWJO        Ty Dolla $ign   
628  2LskIZrCeLxRvCiGP8gxlh         XXXTENTACION   
629  39QJ7KkN3L8LLqQqsO85W1    Yo Yo Honey Singh   
630  1IIKrJVP1C9N7iPtG6eOsK           Young Thug   

                                            album_name  \
0                                          I NEVER DIE   
1                                   Dancing in My Room   
2                                   TROUBLED MEMORIES!   
3                                  Youngblood (Deluxe)   
4                            13 Reasons Why (Season 3)   
..                         

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score


le = LabelEncoder()

df_encoded = df.copy()

df_encoded['release_date'] = pd.to_datetime(df_encoded['Release Date'], format='%Y-%m-%d')
df_encoded['release_year'] = df_encoded['release_date'].dt.year
df_encoded['track_genre'] = le.fit_transform(df_encoded['track_genre'])


df_encoded.to_csv("fields.csv",index=False)

features = [
    'explicit', 'danceability', 'energy', 'loudness', 'speechiness',
    'acousticness', 'instrumentalness', 'tempo', 'time_signature', 'release_year'
] + [col for col in df_encoded.columns if 'track_genre' in col]  # Include one-hot encoded track genres

X = df_encoded[features]
y = df_encoded[['Performs well on Spotify', 'Performs well on TikTok', 'Performs well on YouTube']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(random_state=42)
multi_target_rf = MultiOutputClassifier(rf, n_jobs=-1)

multi_target_rf.fit(X_train, y_train)

y_pred = multi_target_rf.predict(X_test)

print("Accuracy for 'Performs well on Spotify':", accuracy_score(y_test['Performs well on Spotify'], y_pred[:, 0]))
print("Accuracy for 'Performs well on TikTok':", accuracy_score(y_test['Performs well on TikTok'], y_pred[:, 1]))
print("Accuracy for 'Performs well on YouTube':", accuracy_score(y_test['Performs well on YouTube'], y_pred[:, 2]))


Accuracy for 'Performs well on Spotify': 0.7086614173228346
Accuracy for 'Performs well on TikTok': 0.7007874015748031
Accuracy for 'Performs well on YouTube': 0.7637795275590551


In [21]:
from sklearn.svm import SVC

svm = SVC(kernel='linear', random_state=42)
multi_target_svm = MultiOutputClassifier(svm, n_jobs=-1)

multi_target_svm.fit(X_train, y_train)
y_pred = multi_target_svm.predict(X_test)

print("Accuracy for 'Performs well on Spotify':", accuracy_score(y_test['Performs well on Spotify'], y_pred[:, 0]))
print("Accuracy for 'Performs well on TikTok':", accuracy_score(y_test['Performs well on TikTok'], y_pred[:, 1]))
print("Accuracy for 'Performs well on YouTube':", accuracy_score(y_test['Performs well on YouTube'], y_pred[:, 2]))


Accuracy for 'Performs well on Spotify': 0.6220472440944882
Accuracy for 'Performs well on TikTok': 0.6377952755905512
Accuracy for 'Performs well on YouTube': 0.7007874015748031


In [22]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter=1000, random_state=42)
multi_target_log_reg = MultiOutputClassifier(log_reg, n_jobs=-1)

multi_target_log_reg.fit(X_train, y_train)
y_pred = multi_target_log_reg.predict(X_test)

print("Accuracy for 'Performs well on Spotify':", accuracy_score(y_test['Performs well on Spotify'], y_pred[:, 0]))
print("Accuracy for 'Performs well on TikTok':", accuracy_score(y_test['Performs well on TikTok'], y_pred[:, 1]))
print("Accuracy for 'Performs well on YouTube':", accuracy_score(y_test['Performs well on YouTube'], y_pred[:, 2]))


Accuracy for 'Performs well on Spotify': 0.5511811023622047
Accuracy for 'Performs well on TikTok': 0.5748031496062992
Accuracy for 'Performs well on YouTube': 0.6614173228346457


In [23]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'estimator__n_estimators': [5, 10, 20],
    'estimator__learning_rate': [0.01, 0.1, 0.2],
    'estimator__max_depth': [3, 5, 7],
    'estimator__subsample': [0.6, 0.8, 1.0],
    'estimator__colsample_bytree': [0.6, 0.8, 1.0]
}

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
multi_target_xgb = MultiOutputClassifier(xgb, n_jobs=-1)

grid_search = GridSearchCV(
    estimator=multi_target_xgb,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("Accuracy for 'Performs well on Spotify':", accuracy_score(y_test['Performs well on Spotify'], y_pred[:, 0]))
print("Accuracy for 'Performs well on TikTok':", accuracy_score(y_test['Performs well on TikTok'], y_pred[:, 1]))
print("Accuracy for 'Performs well on YouTube':", accuracy_score(y_test['Performs well on YouTube'], y_pred[:, 2]))

Fitting 3 folds for each of 243 candidates, totalling 729 fits
Best Parameters: {'estimator__colsample_bytree': 0.8, 'estimator__learning_rate': 0.2, 'estimator__max_depth': 3, 'estimator__n_estimators': 10, 'estimator__subsample': 0.6}
Accuracy for 'Performs well on Spotify': 0.6692913385826772
Accuracy for 'Performs well on TikTok': 0.6614173228346457
Accuracy for 'Performs well on YouTube': 0.7007874015748031


In [16]:
import pandas as pd

data = {
    'artists': ['Linkin Park'] * 8,
    'album_name': [
        'Iridescent', 'Hybrid Theory (Bonus Edition)', 'Hybrid Theory', 'Hybrid Theory (20th Anniversary Edition)',
        'Hybrid Theory (20th Anniversary Edition)', 'Meteora', 'Hybrid Theory (20th Anniversary Edition)', 'Hybrid Theory (20th Anniversary Edition)'
    ],
    'track_name': [
        'New Divide', 'High Voltage', 'Papercut', 'Crawling', 'Papercut', 'Lying from You', 'Points of Authority', 'One Step Closer'
    ],
    'popularity': [56, 55, 54, 53, 53, 53, 52, 52],
    'duration_ms': [268613, 225106, 184866, 208960, 184866, 175373, 200240, 157333],
    'explicit': [False, False, False, False, False, False, False, False],
    'danceability': [0.493, 0.593, 0.524, 0.581, 0.524, 0.535, 0.569, 0.492],
    'energy': [0.808, 0.967, 0.939, 0.699, 0.94, 0.951, 0.97, 0.969],
    'key': [5, 7, 9, 4, 9, 3, 8, 9],
    'loudness': [-3.365, -4.26, -3.994, -5.564, -3.994, -3.324, -3.886, -4.419],
    'mode': [0, 0, 1, 1, 1, 0, 0, 1],
    'speechiness': [0.0362, 0.182, 0.113, 0.0337, 0.111, 0.18, 0.196, 0.0492],
    'acousticness': [0.000235, 0.105, 0.00022, 0.0455, 0.000217, 0.00777, 0.03, 0.00144],
    'instrumentalness': [0, 0, 0, 0, 0, 0, 0, 0.00000136],
    'liveness': [0.0983, 0.314, 0.623, 0.536, 0.623, 0.0669, 0.0993, 0.0789],
    'valence': [0.38, 0.287, 0.661, 0.282, 0.664, 0.723, 0.435, 0.538],
    'tempo': [117.971, 93.916, 150.184, 105.074, 150.184, 160.102, 95.056, 95.137],
    'time_signature': [4] * 8,
    'track_genre': [23] * 8,
}

df = pd.DataFrame(data)
df['release_year'] = [2010, 2000, 2000, 2000, 2000, 2003, 2000, 2000]

features = [
    'explicit', 'danceability', 'energy', 'loudness', 'speechiness',
    'acousticness', 'instrumentalness', 'tempo', 'time_signature', 'release_year'
] + ['track_genre']

X_new = df[features]
y_new_pred = multi_target_rf.predict(X_new)

print("Predictions for new data:")
print("Performs well on Spotify:", y_new_pred[:, 0])
print("Performs well on TikTok:", y_new_pred[:, 1])
print("Performs well on YouTube:", y_new_pred[:, 2])

Predictions for new data:
Performs well on Spotify: [1 1 1 1 1 1 1 1]
Performs well on TikTok: [0 0 0 0 0 0 0 0]
Performs well on YouTube: [0 0 0 0 0 0 0 0]
